# Ensemble Learning

## Initial Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features
# I am trying to predict loan_status so x becomes everything minus loan_status 
X = df.drop(columns =['loan_status'])
# getting dummie variables (numerical data) for columns with strings 
X  =pd.get_dummies(X)
# Create our target
y = df.loan_status

In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [7]:
# Check the balance of our target values 
# balance for y variable 
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
# comparing against my trained data to see the difference 
X.shape

(68817, 95)

In [9]:
# library needed to split X and y into test and train 
from sklearn.model_selection import train_test_split
# Split the X and y into X_train, X_test, y_train, y_test
X_train,X_test, y_train, y_test = train_test_split(X,y)
# looking at my train data 
X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# importing the StandardScaler library 
from sklearn.preprocessing import StandardScaler
# Create the StandardScaler instance
scaler = StandardScaler()
scaler

StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_train_scaled = scaler.fit(X_train)
# no need to fit the y_train because it is already in a small scale 

In [12]:
# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
from sklearn.ensemble import RandomForestClassifier
# n_estimators = the number of trees in the forest 
rf_model = RandomForestClassifier(n_estimators = 100,random_state = 1)
# Resample the training data with the BalancedRandomForestClassifier
rf_model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=1)

In [14]:
# to calculate the balanced accuracy score I need to have a prediction variable 
ypredictions = rf_model.predict(X_test_scaled)
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test,ypredictions)

0.6521762199751298

In [15]:
# Display the confusion matrix
confusion_matrix(y_test,ypredictions)

array([[   25,    57],
       [    9, 17114]])

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,ypredictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.74      0.30      1.00      0.43      0.55      0.28        82
   low_risk       1.00      1.00      0.30      1.00      0.55      0.33     17123

avg / total       1.00      1.00      0.31      1.00      0.55      0.33     17205



In [17]:
# List the features sorted in descending order by feature importance
# Calculating feature importance 
importance_list = rf_model.feature_importances_
importance_list

array([0.01752928, 0.01306889, 0.02186014, 0.01435339, 0.01558977,
       0.0037112 , 0.00509365, 0.00955259, 0.00205099, 0.01432079,
       0.01254033, 0.01839834, 0.02118914, 0.07353943, 0.06845368,
       0.07411898, 0.07037848, 0.01377355, 0.        , 0.        ,
       0.06941115, 0.00085631, 0.        , 0.        , 0.00416965,
       0.01375373, 0.00648282, 0.00670209, 0.00432163, 0.00622224,
       0.01197614, 0.01171542, 0.01316152, 0.00565234, 0.00715903,
       0.01339766, 0.01170761, 0.01293688, 0.00626556, 0.00696341,
       0.00833004, 0.00825826, 0.01312118, 0.01300522, 0.01278838,
       0.00031914, 0.        , 0.0126492 , 0.01493146, 0.00933423,
       0.0067593 , 0.00523122, 0.00953256, 0.00934582, 0.00277835,
       0.00878795, 0.00989196, 0.00633346, 0.00825092, 0.00943803,
       0.00778264, 0.01025286, 0.00828339, 0.00844363, 0.        ,
       0.        , 0.0014859 , 0.00607501, 0.00770985, 0.00595256,
       0.00193377, 0.        , 0.01206131, 0.01285603, 0.01126

In [18]:
# sorting the feature importances in descending order
#zip gives allows us to enter the labels for the feature importance sort list
sorted(zip(rf_model.feature_importances_,X.columns[:10]),reverse = True)

[(0.021860140726261176, 'installment'),
 (0.01752928103174526, 'loan_amnt'),
 (0.015589770494225717, 'dti'),
 (0.014353393398631988, 'annual_inc'),
 (0.01432078953810392, 'revol_bal'),
 (0.01306889294664896, 'int_rate'),
 (0.009552590408585264, 'open_acc'),
 (0.005093649372411004, 'inq_last_6mths'),
 (0.0037111988224175878, 'delinq_2yrs'),
 (0.002050987317708005, 'pub_rec')]

### Easy Ensemble Classifier

In [19]:
# importing appropriate library 
from imblearn.ensemble import EasyEnsembleClassifier
# Train the Classifier
classifier = EasyEnsembleClassifier(n_estimators = 100,random_state =1)
# fitting the model 
classifier.fit(X_train_scaled,y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [20]:
# getting prediction variable 
predictions = classifier.predict(X_test_scaled)
# Calculated the balanced accuracy score
bca = balanced_accuracy_score(y_test, predictions)
print(f"The balanced accuracy score is: {bca}")

The balanced accuracy score is: 0.8946556692396335


In [21]:
# Display the confusion matrix
confusion_matrix(y_test,predictions)

array([[   69,    13],
       [  893, 16230]])

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.84      0.95      0.13      0.89      0.79        82
   low_risk       1.00      0.95      0.84      0.97      0.89      0.81     17123

avg / total       0.99      0.95      0.84      0.97      0.89      0.81     17205



### Final Q & A's 

1. Which model had the best balanced accuracy score?

   The random forest classifier has the best balanced accurary score of the two models.

2. Which model had the best recall score?

    The random forest classifier has the best recall score. 

3. Which model had the best geometric mean score?

    The the esay ensemble classifer model has the best geometric mean score. 

4. What are the top three features?

    The top three features are installment, dti and loan amount. 